# LocalizadorOfis

In [1]:
# Imports
import sys  
import os
sys.path.append(os.getcwd() + '/src')
from query_functions import contadorPOIs, mostradorPOIs
import pandas as pd
from pandas import json_normalize
import requests
import json
import folium
import re
# Connect with the database
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/datamad0320")
db = client.get_database()

## Companies in USA with the category *design*

In [2]:
# Create the query
query = {"$and":[{"offices.country_code": "USA"}, {"category_code": "design"}]}
# Do the query. Extract name company and where is located.
results = (list(db.Crunch.find((query), {"name", "offices.city", "offices.country_code"})))
print(len(results))
for e in results:
    print(e["name"],"-->", e["offices"][0]["city"])

3
99designs --> San Francisco
Graticle --> Ellensburg
Kickstarter --> Brooklyn


The search will be focused in those cities. To choose one, we are going to see which city has the largest number of companies which have raised at least 1 Million of dollars (as the developers asked for).

In [3]:
# Limit the search to 3 cities: San Francisco, Ellensburg and Brooklyn
# Rank the cities with more companies having raised at least 1 Million dollars
# Function to count the number of companies 
def millionDollar(ciudad):
    # Regex
    match = re.compile(r'[M]')
    # Create the query
    query = {"$and":[{"offices.city": ciudad}, {"total_money_raised": match}]}
    # Do the query. Extract name company and where is located.
    results = (list(db.Crunch.find((query), {"name", "offices.city", "offices.country_code"})))
    return (len(results))

for ciudad in ['San Francisco', 'Ellensburg', 'Brooklyn']:
    print(ciudad,":", millionDollar(ciudad))

San Francisco : 433
Ellensburg : 0
Brooklyn : 8


According to this result, San Francisco is the city that we are looking for.

## Offices close to a design company
Now, let's look for office locations close to the *99designs*, the only design company in San Francisco.

In [4]:
design_office = list(db.companies_prepared.find({"name": "99designs"}, {"name", "office"}))
design_office

[{'_id': ObjectId('5e99bbd2b8b3684df7d5acdc'),
  'name': '99designs',
  'office': {'type': 'Point', 'coordinates': [144.9868546, -37.8026587]}},
 {'_id': ObjectId('5e99bbd2b8b3684df7d5acdd'),
  'name': '99designs',
  'office': {'type': 'Point', 'coordinates': [-122.4005983, 37.7955307]}},
 {'_id': ObjectId('5e99bbd2b8b3684df7d5acde'),
  'name': '99designs',
  'office': {'type': 'Point', 'coordinates': [13.4469031, 52.4986199]}}]

 We are interested in the company located in San Francisco, which is the second one. We need its coordinates.

In [5]:
design_coords = design_office[1]["office"]["coordinates"]
design_coords

[-122.4005983, 37.7955307]

Look for the offices within 500 m of this office.

In [6]:
# Defining the query
query = {'office': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': design_coords},
    '$maxDistance': 500}}}

offices_candidates = list(db.companies_prepared.find(query))
len(offices_candidates)

75

## Points of interest closed to the desired office
### Starbucks
Let's look for offices having a Starbucks closer than 100 m. 

In [7]:
# Run the function for each candidate office and store the number of Starbucks found in a list
n_starbucks = [contadorPOIs(offices_candidates[i]["office"]["coordinates"],100, "starbucks") for i in range(len(offices_candidates))]
#print(n_starbucks)

### Vegan restaurants
The CEO is vegan, so nobody wants problems at lunch time. Let's search vegan restautants closer than 200 m.

In [8]:
# Run the function for each candidate office and store the number of vegan restaurants found
n_vegan = [contadorPOIs(offices_candidates[i]["office"]["coordinates"], 150, "vegan") for i in range(len(offices_candidates))]
#print(n_vegan)   

### Night life

Let's look for offices having a pub or similar closer than 50 m. It's very important to avoid the dispersion of the people looking for bars.

In [9]:
# Run the function for each candidate office and store the number of bars/pubs found
n_nightlife = [contadorPOIs(offices_candidates[i]["office"]["coordinates"], 50, "nightlife") for i in range(len(offices_candidates))]
#print(n_nightlife) 

### Schools
Finally, let's think about in the children (and their parents). Look for the offices within 200 m of the nearest school (preschool, elementary school, middle school and highschool).

In [10]:
# Run the function for each candidate office and store the number of bars/pubs found
n_schools = [contadorPOIs(offices_candidates[i]["office"]["coordinates"], 200, "schools") for i in range(len(offices_candidates))]
#print(n_schools) 

### Summarizing the results
It's time to analyze which office(s) achieve the most of the conditions. First, we're goint to create lists to store the offices and its coordinates, and then, join them in a dataframe with the rest of lists.

In [11]:
lista_ofis = [o["name"] for o in offices_candidates]
coord_ofis = [o["office"]["coordinates"] for o in offices_candidates]

# Create a dataframe with the generated lists
lista = [lista_ofis, coord_ofis, n_starbucks, n_vegan, n_nightlife, n_schools]
df = pd.DataFrame([lista_ofis, coord_ofis, n_starbucks, n_vegan, n_nightlife, n_schools],
                  index=['Name', 'Coords', 'N_starbucks', 'N_vegan', 'N_nightlife', 'N_schools']).T

# Extract the coordinates
df["Coords"] = df["Coords"].astype(str)
df["Coords"] = df["Coords"].str.lstrip("[").str.rstrip("]")
df["lon"] = df["Coords"].str.extract(r'(-\d*.\d*.),')
df["lat"] = df["Coords"].str.extract(r'\s(\d*.\d*)')

# Sum the numbers of near points of interest for each office
df["total"] = df.N_starbucks + df.N_vegan + df.N_nightlife + df.N_schools
# Delete the coords column
df = df.drop(["Coords"], axis=1)
# Sort
df.sort_values("total", ascending = False)

,Name,N_starbucks,N_vegan,N_nightlife,N_schools,lon,lat,total
53,Loomia,0,1,3,5,-122.404869,37.796396,9
52,Pulse Entertainment,0,1,3,5,-122.404869,37.796396,9
9,Netbooks,1,2,3,2,-122.40195,37.79471,8
13,IBWIRE Inc,1,2,3,2,-122.4030739,37.7944822,8
28,PopJax,0,1,1,5,-122.403755,37.796216,7
...,...,...,...,...,...,...,...,...
14,Zentact,0,0,0,0,-122.397901,37.794891,0
38,GoodGuide,0,0,0,0,-122.4000032,37.7983181,0
36,AppNinjas,0,0,0,0,-122.4000032,37.7983181,0
35,GoodGuide,0,0,0,0,-122.4000032,37.7983181,0


The office locations that meet the most of the conditions are those corresponding to rows **9** and **13**.

## Mapping the location of selected offices: Office 9 and Office 13
Finally, we are going to present the location of the selected offices, besides their nearest interest points: Starbucks, vegan restaurants, pubs&bars and schools.

In [12]:
# Starbucks
n_starbucks = [mostradorPOIs(offices_candidates[i]["office"]["coordinates"],100, "starbucks") for i in [9,13]]
n_starbucks

[[{'_id': ObjectId('5e9d6fc051d0da6dfcca8238'),
   'object': {'coordinates': [-122.40268, 37.794613]}}],
 [{'_id': ObjectId('5e9d6fc051d0da6dfcca8238'),
   'object': {'coordinates': [-122.40268, 37.794613]}}]]

In [13]:
# Vegan
n_vegan = [mostradorPOIs(offices_candidates[i]["office"]["coordinates"],150, "vegan") for i in [9,13]]
n_vegan

[[{'_id': ObjectId('5e9d99b6ec95185608038c09'),
   'object': {'coordinates': [-122.4016746709, 37.7948202222]}},
  {'_id': ObjectId('5e9d99b6ec95185608038c17'),
   'object': {'coordinates': [-122.4015493596, 37.7942818704]}}],
 [{'_id': ObjectId('5e9d99b6ec95185608038c09'),
   'object': {'coordinates': [-122.4016746709, 37.7948202222]}},
  {'_id': ObjectId('5e9d99b6ec95185608038c17'),
   'object': {'coordinates': [-122.4015493596, 37.7942818704]}}]]

In [14]:
# Nightlife
n_nightlife = [mostradorPOIs(offices_candidates[i]["office"]["coordinates"],50, "nightlife") for i in [9,13]]
n_nightlife

[[{'_id': ObjectId('5e9d6d6d96bf9061424e6e82'),
   'object': {'coordinates': [-122.4019542632, 37.7947801333]}},
  {'_id': ObjectId('5e9d6d7789875968f546ae94'),
   'object': {'coordinates': [-122.4016746709, 37.7948202222]}},
  {'_id': ObjectId('5e9d6d2a0b4aa1f5a688f945'),
   'object': {'coordinates': [-122.4016746709, 37.7948202222]}}],
 [{'_id': ObjectId('5e9d6d6d96bf9061424e6e4d'),
   'object': {'coordinates': [-122.4028219033, 37.794432603]}},
  {'_id': ObjectId('5e9d6d6d96bf9061424e6e46'),
   'object': {'coordinates': [-122.402808934, 37.794462968]}},
  {'_id': ObjectId('5e9d6d6d96bf9061424e6e85'),
   'object': {'coordinates': [-122.4029180332, 37.7942852971]}}]]

In [15]:
# Schools
n_schools = [mostradorPOIs(offices_candidates[i]["office"]["coordinates"],200, "schools") for i in [9,13]]
n_schools

[[{'_id': ObjectId('5e9dc525a399dba995a7a4fd'),
   'object': {'coordinates': [-122.4040004611, 37.7947517878]}},
  {'_id': ObjectId('5e9dc525a399dba995a7a4fc'),
   'object': {'coordinates': [-122.4042043521, 37.7947566637]}}],
 [{'_id': ObjectId('5e9dc525a399dba995a7a4fd'),
   'object': {'coordinates': [-122.4040004611, 37.7947517878]}},
  {'_id': ObjectId('5e9dc525a399dba995a7a4fc'),
   'object': {'coordinates': [-122.4042043521, 37.7947566637]}}]]

Finally, the map

In [16]:
basemap = folium.Map(
    location=[37.7944822, -122.4030739],
    tiles='Stamen Toner',
    zoom_start=18)
# Markers for offices
folium.Marker([df.loc[9,["lat"]], df.loc[9,["lon"]]], popup='Office <b>9</b>').add_to(basemap)
folium.Marker([df.loc[13,["lat"]], df.loc[13,["lon"]]], popup='Office <b>13</b>').add_to(basemap)
# Marker for Starbucks
folium.Marker(((n_starbucks[0][0]['object']['coordinates'][1]), (n_starbucks[0][0]['object']['coordinates'][0])), popup='Starbucks',icon=folium.Icon(color="darkgreen",icon="coffee", prefix='fa')).add_to(basemap)
# Markers for vegan restaurants
folium.Marker(((n_vegan[0][0]['object']['coordinates'][1]), (n_vegan[0][0]['object']['coordinates'][0])), popup='Vegan restaurant', icon=folium.Icon(color="orange",icon="pepper-hot", prefix='fa')).add_to(basemap)
folium.Marker(((n_vegan[0][1]['object']['coordinates'][1]), (n_vegan[0][1]['object']['coordinates'][0])), popup='Vegan restaurant', icon=folium.Icon(color="orange",icon="pepper-hot", prefix='fa')).add_to(basemap)
# Markers for night life
folium.Marker(((n_nightlife[0][0]['object']['coordinates'][1]), (n_nightlife[0][0]['object']['coordinates'][0])), popup='Night Life',icon=folium.Icon(color="red",icon="beer", prefix='fa')).add_to(basemap)
folium.Marker(((n_nightlife[0][1]['object']['coordinates'][1]), (n_nightlife[0][1]['object']['coordinates'][0])), popup='Night Life',icon=folium.Icon(color="red",icon="beer", prefix='fa')).add_to(basemap)
folium.Marker(((n_nightlife[0][2]['object']['coordinates'][1]), (n_nightlife[0][2]['object']['coordinates'][0])), popup='Night Life',icon=folium.Icon(color="red",icon="beer", prefix='fa')).add_to(basemap)
folium.Marker(((n_nightlife[1][0]['object']['coordinates'][1]), (n_nightlife[1][0]['object']['coordinates'][0])), popup='Night Life',icon=folium.Icon(color="red",icon="beer", prefix='fa')).add_to(basemap)
folium.Marker(((n_nightlife[1][1]['object']['coordinates'][1]), (n_nightlife[1][1]['object']['coordinates'][0])), popup='Night Life',icon=folium.Icon(color="red",icon="beer", prefix='fa')).add_to(basemap)
folium.Marker(((n_nightlife[1][2]['object']['coordinates'][1]), (n_nightlife[1][2]['object']['coordinates'][0])), popup='Night Life',icon=folium.Icon(color="red",icon="beer", prefix='fa')).add_to(basemap)
# Markers for schools
folium.Marker(((n_schools[0][0]['object']['coordinates'][1]), (n_schools[0][0]['object']['coordinates'][0])), popup='School',icon=folium.Icon(color="darkblue",icon="graduation-cap", prefix='fa')).add_to(basemap)
folium.Marker(((n_schools[0][1]['object']['coordinates'][1]), (n_schools[0][1]['object']['coordinates'][0])), popup='School',icon=folium.Icon(color="darkblue",icon="graduation-cap", prefix='fa')).add_to(basemap)
basemap